In [1]:
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI

# read local .env file
# find_dotenv 函数通常用于搜索和定位目录树中的 .env 文件，而 load_dotenv 函数用于将 .env 文件中的变量加载到环境中。
_ = load_dotenv(find_dotenv())

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)
# embeddings = OpenAIEmbeddings()

In [2]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

"""
调用语言模型之后的下一步是对语言模型进行一系列调用。我们可以使用序列链来实现这一点，序列链是按照预定义的顺序执行链接的链。
具体来说，我们将使用 SimpleSequentialChain。这是最简单的顺序链类型，其中每个步骤都有一个输入/输出，一个步骤的输出是下一个步骤的输入。

在本教程中，我们的顺序链将:

首先，为产品创建一个公司名称。我们将重用之前初始化的 LLMChain 来创建这个公司名称。

然后，为产品创建一个口号。我们将初始化一个新的 LLMChain 来创建这个口号，如下所示。
"""
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)

human_message_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        template="What is a good name for a company that makes {product}?",
        input_variables=["product"],
    )
)
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
chain = LLMChain(llm=llm, prompt=chat_prompt_template)

second_prompt = PromptTemplate(
    input_variables=["company_name"],
    template="Write a catchphrase for the following company: {company_name}",
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)


In [4]:
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[chain, chain_two], verbose=True)

# Run the chain specifying only the input variable for the first chain.
catchphrase = overall_chain.invoke("colorful socks")
print(catchphrase)




> Entering new SimpleSequentialChain chain...
VibrantSocks
"Step up your style game with VibrantSocks - Where every step is a statement!"

> Finished chain.
{'input': 'colorful socks', 'output': '"Step up your style game with VibrantSocks - Where every step is a statement!"'}


In [ ]:
""" output


[1m> Entering new SimpleSequentialChain chain...[0m
[36;1m[1;3mVibrantSocks[0m
[33;1m[1;3m"Step up your style game with VibrantSocks - Where every step is a statement!"[0m

[1m> Finished chain.[0m
{'input': 'colorful socks', 'output': '"Step up your style game with VibrantSocks - Where every step is a statement!"'}

"""

In [5]:
"""
SequentialChain，允许多个输入/输出
"""
# This is an LLMChain to write a synopsis given a title of a play and the era it is set in.
template = """You are a playwright. Given the title of play and the era it is set in, it is your job to write a synopsis for that title.
 
Title: {title}
Era: {era}
Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title", 'era'], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="synopsis")


In [6]:
# This is an LLMChain to write a review of a play given a synopsis.
template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.
 
Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review")


In [7]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True)
 

In [8]:
overall_chain.invoke({"title":"Tragedy at sunset on the beach", "era": "Victorian England"})
 



> Entering new SequentialChain chain...

> Finished chain.


{'title': 'Tragedy at sunset on the beach',
 'era': 'Victorian England',
 'synopsis': 'In the grandeur of Victorian England, "Tragedy at Sunset on the Beach" unfolds against the backdrop of a picturesque coastal town. The play opens with a vibrant and bustling beach scene, where society\'s elite gather to enjoy the last rays of the setting sun.\n\nAmidst the opulent atmosphere, we meet the wealthy and influential Pemberton family. Lord Pemberton, a respected nobleman, and his elegant wife, Lady Amelia, are hosting a lavish beach party to celebrate their daughter, Victoria\'s engagement to the charming and ambitious Mr. Edward Sinclair.\n\nAs the evening progresses, tensions simmer beneath the surface, revealing the complex dynamics within the Pemberton family. Lady Amelia, burdened by societal expectations, struggles to maintain her composure while concealing a dark secret. Lord Pemberton, haunted by his past, finds solace in the bottom of a glass, oblivious to the turmoil unfolding ar

In [ ]:
""" output


[1m> Entering new SequentialChain chain...[0m

[1m> Finished chain.[0m

{'title': 'Tragedy at sunset on the beach',
 'era': 'Victorian England',
 'synopsis': 'In the grandeur of Victorian England, "Tragedy at Sunset on the Beach" unfolds against the backdrop of a picturesque coastal town. The play opens with a vibrant and bustling beach scene, where society\'s elite gather to enjoy the last rays of the setting sun.\n\nAmidst the opulent atmosphere, we meet the wealthy and influential Pemberton family. Lord Pemberton, a respected nobleman, and his elegant wife, Lady Amelia, are hosting a lavish beach party to celebrate their daughter, Victoria\'s engagement to the charming and ambitious Mr. Edward Sinclair.\n\nAs the evening progresses, tensions simmer beneath the surface, revealing the complex dynamics within the Pemberton family. Lady Amelia, burdened by societal expectations, struggles to maintain her composure while concealing a dark secret. Lord Pemberton, haunted by his past, finds solace in the bottom of a glass, oblivious to the turmoil unfolding around him.\n\nMeanwhile, Victoria, torn between duty and her own desires, grapples with the weight of her impending marriage. Her heart yearns for a simpler life, away from the constraints of high society, and she finds solace in the company of a mysterious artist who frequents the beach.\n\nAs the sun dips below the horizon, tragedy strikes, shattering the illusion of the perfect evening. A shocking revelation threatens to unravel the carefully constructed façade of the Pemberton family, exposing their deepest secrets and hidden desires. The once idyllic beach becomes a stage for betrayal, heartbreak, and ultimately, redemption.\n\n"Tragedy at Sunset on the Beach" is a gripping tale of love, sacrifice, and the consequences of societal expectations. Set against the backdrop of Victorian England, this play delves into the complexities of human nature, exploring the fragility of relationships and the lengths one will go to protect their reputation. As the sun sets on this tragic tale, the audience is left pondering the true cost of societal conformity and the pursuit of happiness.',
 'review': '"Tragedy at Sunset on the Beach" is a captivating and thought-provoking play that transports audiences to the grandeur of Victorian England. Against the picturesque backdrop of a coastal town, this production skillfully explores the intricate dynamics within the wealthy and influential Pemberton family.\n\nFrom the moment the curtain rises, the audience is immersed in a vibrant beach scene, filled with the bustling energy of society\'s elite. The set design and costumes beautifully capture the opulence of the era, transporting viewers to a time of lavish beach parties and societal expectations.\n\nThe strength of this play lies in its ability to delve into the complexities of human nature. Lady Amelia, portrayed with great depth and vulnerability, grapples with the burden of societal expectations while concealing a dark secret. Her performance is a standout, capturing the internal struggle of a woman torn between duty and personal desires.\n\nLord Pemberton, haunted by his past, is portrayed with equal nuance. The actor skillfully conveys the character\'s descent into alcoholism, finding solace in the bottom of a glass while remaining oblivious to the turmoil unfolding around him. The portrayal is both heartbreaking and compelling, showcasing the destructive power of unresolved trauma.\n\nThe central conflict of the play revolves around Victoria, the Pemberton\'s daughter, brilliantly portrayed by a talented young actress. Her struggle between societal expectations and her own desires is palpable, and the audience is drawn into her emotional journey. The chemistry between Victoria and the mysterious artist adds an intriguing layer to the narrative, highlighting the tension between societal conformity and personal fulfillment.\n\nThe climax of the play is truly gripping, as a shocking revelation threatens to unravel the carefully constructed façade of the Pemberton family. The tension builds to a crescendo, and the once idyllic beach becomes a stage for betrayal, heartbreak, and ultimately, redemption. The emotional impact of this moment is heightened by the skillful direction and the exceptional performances of the cast.\n\n"Tragedy at Sunset on the Beach" is a play that lingers in the mind long after the final curtain call. It raises important questions about the true cost of societal conformity and the pursuit of happiness. The themes of love, sacrifice, and the consequences of societal expectations are explored with depth and sensitivity, leaving the audience pondering the complexities of human nature.\n\nOverall, "Tragedy at Sunset on the Beach" is a must-see production that showcases the talent of its cast and creative team. It is a testament to the power of theater to transport and engage audiences, leaving them with a renewed appreciation for the fragility of relationships and the choices we make in the pursuit of happiness.'}
"""

In [9]:
"""
顺序链中的记忆
有时您可能想传递一些上下文以在链的每个步骤或链的后面部分中使用，但维护和链接输入/输出变量可能会很快变得混乱。使用 SimpleMemory 是一种方便的方式来管理这个问题并清理您的链。

举个例子，假设你使用之前介绍过的playwright SequentialChain，你想要在剧本中添加一些关于日期、时间和地点的上下文信息，并且使用生成的简介和评论来创建一些社交媒体发布文本。
你可以将这些新的上下文变量添加为input_variables，或者我们可以添加一个SimpleMemory到链中来管理这个上下文：
"""
from langchain.chains import SequentialChain
from langchain.memory import SimpleMemory
 
template = """You are a social media manager for a theater company. Given the title of play, the era it is set in, the date,time and location, the synopsis of the play, and the review of the play, it is your job to write a social media post for that play.
 
Here is some context about the time and location of the play:
Date and Time: {time}
Location: {location}
 
Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:
{review}
 
Social Media Post:
"""
prompt_template = PromptTemplate(input_variables=["synopsis", "review", "time", "location"], template=template)
social_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="social_post_text")
 
overall_chain = SequentialChain(
    memory=SimpleMemory(memories={"time": "December 25th, 8pm PST", "location": "Theater in the Park"}),
    chains=[synopsis_chain, review_chain, social_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["social_post_text"],
    verbose=True)
 
overall_chain.invoke({"title":"Tragedy at sunset on the beach", "era": "Victorian England"})
 

D:\anaconda3\envs\learn-langchain\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...

> Finished chain.


{'title': 'Tragedy at sunset on the beach',
 'era': 'Victorian England',
 'time': 'December 25th, 8pm PST',
 'location': 'Theater in the Park',
 'social_post_text': '🌅 Don\'t miss the gripping Victorian drama, "Tragedy at Sunset on the Beach," on December 25th at 8pm PST at the Theater in the Park! 🎭 Immerse yourself in a world of opulence, secrets, and forbidden desires as the lives of three families unravel against the backdrop of a serene coastal town. \n\n🌊 Witness the aristocratic Hastings family, known for their wealth and influence, as they navigate a scandalous affair that threatens to shatter their carefully constructed facade. Lady Amelia Hastings finds solace in the arms of a mysterious stranger, setting off a chain of tragic events that will forever change their lives.\n\n🎨 Meanwhile, the humble Fisher family becomes unwittingly embroiled in the chaos, as young Thomas Fisher falls deeply in love with Lady Amelia, unaware of her marital status. Their forbidden love affair te

In [ ]:
""" output


[1m> Entering new SequentialChain chain...[0m

[1m> Finished chain.[0m

{'title': 'Tragedy at sunset on the beach',
 'era': 'Victorian England',
 'time': 'December 25th, 8pm PST',
 'location': 'Theater in the Park',
 'social_post_text': '🌅 Don\'t miss the gripping Victorian drama, "Tragedy at Sunset on the Beach," on December 25th at 8pm PST at the Theater in the Park! 🎭 Immerse yourself in a world of opulence, secrets, and forbidden desires as the lives of three families unravel against the backdrop of a serene coastal town. \n\n🌊 Witness the aristocratic Hastings family, known for their wealth and influence, as they navigate a scandalous affair that threatens to shatter their carefully constructed facade. Lady Amelia Hastings finds solace in the arms of a mysterious stranger, setting off a chain of tragic events that will forever change their lives.\n\n🎨 Meanwhile, the humble Fisher family becomes unwittingly embroiled in the chaos, as young Thomas Fisher falls deeply in love with Lady Amelia, unaware of her marital status. Their forbidden love affair tests their loyalty, integrity, and survival.\n\n🌟 Transport yourself to a bygone era, where societal expectations and strict social norms govern every aspect of life. Reflect on the timeless themes of love, sacrifice, and the destructive power of secrets. \n\n📰 According to The New York Times, "Tragedy at Sunset on the Beach" is a must-see Victorian drama that explores the complexities of human nature. The richly developed characters, evocative setting, and thought-provoking exploration of love and morality make this play a standout in the theatrical landscape.\n\n🎟️ Get your tickets now and experience the grandeur of Victorian England at the Theater in the Park! Don\'t miss out on this gripping tale of love, betrayal, and tragedy. Book your seats today! #TragedyatSunset #VictorianDrama #TheaterExperience'}
"""